In [1]:
import pandas as pd
import numpy as np
import re




In [2]:
def salary_clean(string):
    
    string = str(string)
    if 'year' in string:
        salary_type.append('year')
        try:
            sals = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)', string.replace(',',''))
            if len(sals)==2:
                low = int(sals[0].replace('£','').replace(' ', ''))
                high = int(sals[1].replace('£','').replace(' ', ''))
                return int((low + high)/2)
            elif len(sals)==1:
                return int(sals[0].replace('£','').replace(' ', ''))
        except:
            return string
    
    
    elif 'hour' in string:
        salary_type.append('hour')
        try:
            sals = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)', string)
            if len(sals)==2:
                low = float(sals[0].replace('£',''))
                high = float(sals[1].replace('£',''))
                return ((low + high)/2) * 37.5 * 52
            elif len(sals)==1:
                return float(sals[0].replace('£','').replace(' ', '')) * 37.5 * 52
        except: 
            return string
    
    elif 'month' in string:
        salary_type.append('month')
        try:
            sals = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)', string.replace(',',''))
            if len(sals)==2:
                low = float(sals[0].replace('£',''))
                high = float(sals[1].replace('£',''))
                return ((low + high) / 2) * 12    
            elif len(sals)==1:
                return float(sals[0].replace('£','').replace(' ', '')) * 12
        except: 
            return string
        
    elif 'week' in string:
        salary_type.append('week')
        try:
            sals = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)', string)
            if len(sals)==2:
                low = float(sals[0].replace('£',''))
                high = float(sals[1].replace('£',''))
                return ((low+high) / 2) * 52  
            elif len(sals)==1:
                return float(sals[0].replace('£','').replace(' ', '')) * 52
        except: 
            return string
    
    elif 'day' in string:
        salary_type.append('day')
        try:
            sals = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)', string)
            if len(sals)==2:
                low = float(sals[0].replace('£',''))
                high = float(sals[1].replace('£',''))
                return ((low + high)/2) * 365
            elif len(sals)==1:
                return float(sals[0].replace('£','').replace(' ', '')) * 20 * 12
        except: 
            return string 
        
    elif string == 'None':
        salary_type.append('None')
        return np.nan

    return string

In [12]:
def remove_postcode(location):
    
    try:
        postcode = re.findall(r"""[A-Z]{1,2}\d[A-Z\d]? ?\d[A-Z]{2}""", location)
        if location.replace(postcode[0], '')[-1]==' ':
            return location.replace(postcode[0], '')[:-1]
        else:
            return location.replace(postcode[0], '')
    except:
        return location

In [13]:
city_list = [ 'london', 'reading', 'manchester', 'liverpool',
             'birmingham', 'bristol','cambridge', 'leeds',
             'edinburgh', 'glasgow', 'cardiff','nottingham',
             'newcastle']



In [14]:
columns = ['Unnamed: 0', 'Title', 'Company', 'City',
           'Location', 'Salary','Review', 'Link']

In [15]:
job_dict = {}

for city in city_list:
    
    df = pd.read_csv(
        r'/Users/finnalexander/Desktop/CSV files/Indeed_stats/{}.csv'.format(city))
    
    city_cap = city.capitalize()
    df.insert(loc=3, column='City', value=city_cap)
    
    salary_type = []
    
    
    print('='*40)
    print(city_cap)
    print('-'*5)
    
    df.columns = columns
    
    df['Salary'].fillna('None', inplace=True)

    
    for x in df['Salary'].apply(salary_clean):
        if type(x) == str:
            print('Salary ERROR !!!!')
    
    df.insert(loc=6, column='Paid_by', value=salary_type)
        
    df['Salary'] = df['Salary'].apply(salary_clean)
    
    df.drop('Unnamed: 0', axis=1, inplace=True)
    
    df['Location']=df['Location'].apply(remove_postcode)
    
    df['Location']=df['Location'].apply(lambda x: city_cap if x == 'None' else x)

    job_dict[city] = df

London
-----
Reading
-----
Manchester
-----
Liverpool
-----
Birmingham
-----
Bristol
-----
Cambridge
-----
Leeds
-----
Edinburgh
-----
Glasgow
-----
Cardiff
-----
Nottingham
-----
Newcastle
-----


In [16]:
total_df = pd.concat([job_dict[city] for city in city_list],
                    ignore_index=True)

total_df.head()

,Title,Company,City,Location,Salary,Paid_by,Review,Link
0,Junior Data Analyst,Fairfield School of Business,London,NaN,NaN,None,None,https://www.indeed.co.uk/company/Fairfield-Sch...
1,"Research Assistant, Assessment Research Group",NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=45845cddaf9...
2,2021 Data Insights Internship Programme,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=6f15a97b411...
3,Senior Data Scientist,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=46650666a36...
4,Quantitative Risk Manager,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=bee80722d53...


In [17]:
total_df.shape

(974, 8)

In [18]:
empty_salary_jobs = total_df[total_df['Salary'].isnull()]

In [19]:
dupes = total_df[total_df['Link'].duplicated()].index

In [20]:
dupes

Int64Index([ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
            ...
            858, 867, 869, 886, 888, 891, 893, 895, 896, 898],
           dtype='int64', length=594)

In [21]:
total_df.drop(dupes, inplace=True)

In [22]:
total_df.head()

,Title,Company,City,Location,Salary,Paid_by,Review,Link
0,Junior Data Analyst,Fairfield School of Business,London,NaN,NaN,None,None,https://www.indeed.co.uk/company/Fairfield-Sch...
1,"Research Assistant, Assessment Research Group",NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=45845cddaf9...
2,2021 Data Insights Internship Programme,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=6f15a97b411...
3,Senior Data Scientist,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=46650666a36...
4,Quantitative Risk Manager,NaN,London,NaN,NaN,None,NaN,https://www.indeed.co.uk/rc/clk?jk=bee80722d53...


In [23]:
total_df.dropna()

,Title,Company,City,Location,Salary,Paid_by,Review,Link
6,Pre-Registrant/Clinical Scientist,The Newcastle upon Tyne Hospitals NHS Foundati...,London,Royal Victoria Station,41696.0,year,3.4,https://www.indeed.co.uk/rc/clk?jk=9b0910ef800...
285,Business Intelligence Executive,Oxfam GB,Reading,Oxford,24548.0,year,4.3,https://www.indeed.co.uk/rc/clk?jk=e43c693f373...
286,Insight Analyst,Cosine UK Ltd,Reading,Thame,30000.0,year,3.6,https://www.indeed.co.uk/company/Cosine-UK/job...
291,Postdoctoral Research Scientist,University of Oxford,Reading,Oxford,36569.0,year,4.2,https://www.indeed.co.uk/rc/clk?jk=f0641f13880...
292,Postdoctoral Data Scientist,University of Oxford,Reading,Oxford,36569.0,year,4.2,https://www.indeed.co.uk/rc/clk?jk=c83baa7f644...
...,...,...,...,...,...,...,...,...
965,Higher Statistical Officer across GSS,UK Government - Government Statistical Service,Newcastle,Newcastle upon Tyne,28122.0,year,None,https://www.indeed.co.uk/rc/clk?jk=01a7ea30be1...
966,Senior Statistical Officer across the GSS - Na...,UK Government - Government Statistical Service,Newcastle,Newcastle upon Tyne,35540.0,year,None,https://www.indeed.co.uk/rc/clk?jk=88b053a360e...
967,Senior Statistician x 4,Public Sector Resourcing,Newcastle,Newcastle upon Tyne,155125.0,day,3.0,https://www.indeed.co.uk/company/Public-Sector...
968,Research Associate/Senior Research Associate,Newcastle University,Newcastle,Newcastle upon Tyne,43346.0,year,4.2,https://www.indeed.co.uk/rc/clk?jk=d497e1b47d2...


In [24]:
salary_df = total_df.dropna()

In [25]:
salary_df.to_csv(r'uk_stats_jobs.csv')

In [ ]:
empty_salary_jobs.reset_index(drop=True, inplace=True)